In [1]:
!pip install statsforecast

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from statsforecast.models import (AutoARIMA, HoltWinters)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import OneHotEncoder
from pylab import rcParams
import os
  
# Ignore harmless warnings 
import warnings 
warnings.filterwarnings("ignore") 


StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 66.4 MB/s eta 0:00:00


In [2]:
# Load dataset
konya = pd.read_parquet("abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/konya_features.parquet")

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 4, Finished, Available, Finished)

In [3]:
# vizualize dataset
konya.head()
konya.tail()

# print info
print(konya.info())

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 5, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532614 entries, 0 to 532613
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   store_id               532614 non-null  object        
 1   product_id             532614 non-null  object        
 2   date                   532614 non-null  datetime64[ns]
 3   sales                  532614 non-null  float64       
 4   revenue                532614 non-null  float64       
 5   stock                  532614 non-null  float64       
 6   price                  531602 non-null  float64       
 7   promo_type_1           532614 non-null  category      
 8   promo_bin_1            75780 non-null   category      
 9   promo_type_2           532614 non-null  category      
 10  promo_bin_2            479 non-null     category      
 11  promo_discount_2       479 non-null     float64       
 12  promo_discount_type_2  479 non-null     cate

In [4]:
# change dtypes 

cat_col = ['store_id', 'product_id', 'season', 'week']
konya[cat_col] = konya[cat_col].astype('category')

# Drop columns not necessary
df = konya.copy()
# List of columns to drop
columns_to_drop = [
    'promo_bin_1', 
    'promo_bin_2',
    'revenue', 
    'promo_discount_2', 
    'promo_discount_type_2', 
    'hierarchy2_id', 
    'hierarchy3_id', 
    'hierarchy4_id', 
    'hierarchy5_id', 
    'city_id_old', 
    'country_id', 
    'city_code',
    'product_length', 
    'product_depth',
    'product_width',
    'weekday'
]

# Drop the specified columns
df = df.drop(columns=columns_to_drop, errors='ignore')

#confirm drop
df.info()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 6, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532614 entries, 0 to 532613
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   store_id       532614 non-null  category      
 1   product_id     532614 non-null  category      
 2   date           532614 non-null  datetime64[ns]
 3   sales          532614 non-null  float64       
 4   stock          532614 non-null  float64       
 5   price          531602 non-null  float64       
 6   promo_type_1   532614 non-null  category      
 7   promo_type_2   532614 non-null  category      
 8   cluster_id     532614 non-null  category      
 9   hierarchy1_id  532614 non-null  category      
 10  storetype_id   532614 non-null  category      
 11  store_size     532614 non-null  int64         
 12  season         532614 non-null  category      
 13  week           532614 non-null  category      
 14  holiday        532614 non-null  category      
 15  

In [5]:
# Filter products that are present in 2019
products_in_2019 = df[df['date'].dt.year == 2019]['product_id'].unique()

# Filter the original DataFrame to include only products sold in 2019
df_filtered = df[df['product_id'].isin(products_in_2019)]

# Display the last few rows of the filtered DataFrame
df_filtered.tail()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 7, Finished, Available, Finished)

,store_id,product_id,date,sales,stock,price,promo_type_1,promo_type_2,cluster_id,hierarchy1_id,storetype_id,store_size,season,week,holiday,month_name
532609,S0142,P0733,2019-09-30,0.0,12.0,0.75,PR14,PR03,cluster_7,H00,ST04,31,3,40,N,Sep
532610,S0142,P0741,2019-09-30,0.0,3.0,32.90,PR10,PR03,cluster_0,H01,ST04,31,3,40,N,Sep
532611,S0142,P0742,2019-09-30,0.0,5.0,69.90,PR07,PR03,cluster_0,H01,ST04,31,3,40,N,Sep
532612,S0142,P0747,2019-09-30,0.0,16.0,21.90,PR14,PR03,cluster_0,H01,ST04,31,3,40,N,Sep
532613,S0142,P0748,2019-09-30,0.0,18.0,18.90,PR14,PR03,cluster_0,H01,ST04,31,3,40,N,Sep


In [6]:
# Filter rows where 'unique_id' is 'P0035' that only appears in 2018 to confirm filtering
filtered_df = df_filtered[df_filtered['product_id'] == 'P0531']

print(filtered_df.tail())

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 8, Finished, Available, Finished)

       store_id product_id       date  sales  stock  price promo_type_1  \
452911    S0142      P0531 2018-09-20    0.0   14.0   1.15         PR14   
453135    S0142      P0531 2018-09-21    8.0    6.0   1.15         PR14   
453359    S0142      P0531 2018-09-22    0.0    6.0   1.15         PR14   
453583    S0142      P0531 2018-09-23    1.0    5.0   1.15         PR14   
453807    S0142      P0531 2018-09-24    6.0    0.0   1.15         PR14   

       promo_type_2 cluster_id hierarchy1_id storetype_id  store_size season  \
452911         PR03  cluster_9           H00         ST04          31      3   
453135         PR03  cluster_9           H00         ST04          31      3   
453359         PR03  cluster_9           H00         ST04          31      3   
453583         PR03  cluster_9           H00         ST04          31      3   
453807         PR03  cluster_9           H00         ST04          31      3   

       week holiday month_name  
452911   38       N        Sep  
45

In [7]:
# Get unique store_ids
unique_store_ids = df_filtered['store_id'].unique()

# Create a dictionary of DataFrames, each keyed by store_id
store_datasets = {}
for store_id in unique_store_ids:
    store_datasets[store_id] = df_filtered[df_filtered['store_id'] == store_id].copy()

# dataset for store_id 'S0142'
store_S0142	 = store_datasets['S0142']
print("Store S0142 dataset:")
print(store_S0142.head())

# dataset for store_id S0094
store_S0094	 = store_datasets['S0094']
print("Store S0094 dataset:")
print(store_S0094.head())

# dataset for store_id S0030
store_S0030	 = store_datasets['S0030']
print("Store S0030 dataset:")
print(store_S0030.head())


StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 9, Finished, Available, Finished)

Store S0142 dataset:
       store_id product_id       date  sales  stock  price promo_type_1  \
329162    S0142      P0005 2017-01-02    0.0    2.0  33.90         PR14   
329163    S0142      P0015 2017-01-02    0.0   12.0   2.60         PR14   
329164    S0142      P0017 2017-01-02    0.0   17.0   1.49         PR14   
329165    S0142      P0018 2017-01-02    0.0    2.0   1.95         PR14   
329168    S0142      P0051 2017-01-02    0.0   15.0   0.70         PR14   

       promo_type_2 cluster_id hierarchy1_id storetype_id  store_size season  \
329162         PR03  cluster_9           H03         ST04          31      1   
329163         PR03  cluster_1           H00         ST04          31      1   
329164         PR03  cluster_6           H00         ST04          31      1   
329165         PR03  cluster_4           H00         ST04          31      1   
329168         PR03  cluster_7           H00         ST04          31      1   

       week holiday month_name  
329162    1   

In [8]:
# select store as df
df = store_S0094.copy() # replace according to store


StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 10, Finished, Available, Finished)

In [9]:
# get number of cluster_ids from store

store_S0094['cluster_id'].nunique()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 11, Finished, Available, Finished)

10

## S0094

In [10]:
# Change the granularity of time series data from daily to weekly

# Set 'ds' column as the index
df.set_index('date', inplace=True)
# Sort the DataFrame by the index (dates)
df.sort_index(inplace=True)

# Group by 'product_id' and resample to weekly frequency
df_weekly = df.groupby(['cluster_id']).resample('W-MON').agg({
    'sales': 'sum',  # Aggregate sales data
    'stock': 'max',  # Take the max stock value of the week
    'price': 'mean',  # Last price for the week
    'promo_type_1': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'promo_type_2': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    #'cluster_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'hierarchy1_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'storetype_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'store_size': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'season': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'holiday': lambda x: x.mode().iloc[0] if not x.mode().empty else None, 
    'week': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'month_name': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
}).reset_index()

# Display the weekly aggregated DataFrame
print(df_weekly.head())

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 12, Finished, Available, Finished)

  cluster_id       date  sales  stock      price promo_type_1 promo_type_2  \
0  cluster_0 2017-01-02   49.0  228.0  11.559640         PR14         PR03   
1  cluster_0 2017-01-09  252.0  313.0  11.571862         PR14         PR03   
2  cluster_0 2017-01-16  282.0  268.0  11.804322         PR14         PR03   
3  cluster_0 2017-01-23  372.0  170.0  11.797861         PR14         PR03   
4  cluster_0 2017-01-30  284.0  160.0  11.802834         PR14         PR03   

  hierarchy1_id storetype_id  store_size  season holiday  week month_name  
0           H01         ST04          45       1       N     1        Jan  
1           H01         ST04          45       1       N     2        Jan  
2           H01         ST04          45       1       N     3        Jan  
3           H01         ST04          45       1       N     4        Jan  
4           H01         ST04          45       1       N     5        Jan  


In [11]:
df_weekly.info()
#df.info()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 13, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   cluster_id     1440 non-null   category      
 1   date           1440 non-null   datetime64[ns]
 2   sales          1440 non-null   float64       
 3   stock          1440 non-null   float64       
 4   price          1440 non-null   float64       
 5   promo_type_1   1440 non-null   object        
 6   promo_type_2   1440 non-null   object        
 7   hierarchy1_id  1440 non-null   object        
 8   storetype_id   1440 non-null   object        
 9   store_size     1440 non-null   int64         
 10  season         1440 non-null   int64         
 11  holiday        1440 non-null   object        
 12  week           1440 non-null   int64         
 13  month_name     1440 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(3), int64(3), object(6)
memor

In [12]:
# Sort DataFrame by 'date'
df_weekly = df_weekly.sort_values(by='date')
#df = df.sort_values(by='date')

# Reset index after sorting
df_weekly.reset_index(drop=True, inplace=True)
#df.reset_index(drop=True, inplace=True)

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 14, Finished, Available, Finished)

In [13]:
df_weekly.tail()
#df.tail()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 15, Finished, Available, Finished)

,cluster_id,date,sales,stock,price,promo_type_1,promo_type_2,hierarchy1_id,storetype_id,store_size,season,holiday,week,month_name
1435,cluster_2,2019-09-30,70.0,79.0,14.833333,PR14,PR03,H00,ST04,45,3,N,39,Sep
1436,cluster_1,2019-09-30,21.0,64.0,4.942857,PR14,PR03,H00,ST04,45,3,N,39,Sep
1437,cluster_0,2019-09-30,316.0,400.0,40.489475,PR14,PR03,H01,ST04,45,3,N,39,Sep
1438,cluster_8,2019-09-30,25.0,73.0,11.838571,PR14,PR03,H03,ST04,45,3,N,39,Sep
1439,cluster_9,2019-09-30,104.0,113.0,18.778374,PR14,PR03,H00,ST04,45,3,N,39,Sep


# **ARIMA**

### All time range included

In [14]:
# Adjust column names if needed
df_weekly = df_weekly.rename(columns={
    'cluster_id': 'unique_id',
    'date': 'ds',
    'sales': 'y'
})

#funcao do arima
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()


# Rename variable
data = df_weekly.copy()

df_product = data.copy()

# Ensure 'unique_id' and 'ds' are properly formatted
df_product['unique_id'] = df_product['unique_id'].astype(str)
df_product['ds'] = pd.to_datetime(df_product['ds'])

# Separate sales and date from exogenous variables
data2 = df_product[['ds', 'unique_id', 'y']]

# Split data in train for different time-range
train = data2.loc[data2['ds'] < '2019-10-01']
# Count the occurrences of each product
product_counts = train['unique_id'].value_counts()
# Filter products that appear at least 5 times (to not arise error in crossvalidation)
products_to_keep = product_counts[product_counts >= 5].index
train = train[train['unique_id'].isin(products_to_keep)]



# Define AutoArima model
model = [AutoARIMA(season_length=52)]
#Forecast
sf = StatsForecast(models=model, freq='W-MON', n_jobs=-1)

#Cross-validation
crossvalidation_df = sf.cross_validation(
    df = train,
    h = 2,
    step_size = 2,
    n_windows = 3, 
    level = [90]
  )

print("crossvalidation_df:")
print(crossvalidation_df.head())
p = crossvalidation_df 
p = p.reset_index()
p.rename(columns={'index': 'unique_id'}, inplace=True)


print (p.head())

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 16, Finished, Available, Finished)

crossvalidation_df:
                  ds     cutoff      y   AutoARIMA  AutoARIMA-lo-90  \
unique_id                                                             
cluster_0 2019-08-26 2019-08-19  487.0  364.012634       251.943298   
cluster_0 2019-09-02 2019-08-19  295.0  364.012634       236.594315   
cluster_0 2019-09-09 2019-09-02  413.0  357.348145       243.213776   
cluster_0 2019-09-16 2019-09-02  279.0  357.348145       227.800583   
cluster_0 2019-09-23 2019-09-16  171.0  329.708374       215.193787   

           AutoARIMA-hi-90  
unique_id                   
cluster_0       476.081940  
cluster_0       491.430939  
cluster_0       471.482513  
cluster_0       486.895691  
cluster_0       444.222992  
   unique_id         ds     cutoff      y   AutoARIMA  AutoARIMA-lo-90  \
0  cluster_0 2019-08-26 2019-08-19  487.0  364.012634       251.943298   
1  cluster_0 2019-09-02 2019-08-19  295.0  364.012634       236.594315   
2  cluster_0 2019-09-09 2019-09-02  413.0  357.348145    

In [15]:
p.head(10)

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 17, Finished, Available, Finished)

,unique_id,ds,cutoff,y,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,cluster_0,2019-08-26,2019-08-19,487.0,364.012634,251.943298,476.081940
1,cluster_0,2019-09-02,2019-08-19,295.0,364.012634,236.594315,491.430939
2,cluster_0,2019-09-09,2019-09-02,413.0,357.348145,243.213776,471.482513
3,cluster_0,2019-09-16,2019-09-02,279.0,357.348145,227.800583,486.895691
4,cluster_0,2019-09-23,2019-09-16,171.0,329.708374,215.193787,444.222992
5,cluster_0,2019-09-30,2019-09-16,316.0,329.708374,200.090759,459.326019
6,cluster_1,2019-08-26,2019-08-19,26.0,28.753464,8.387361,49.119568
7,cluster_1,2019-09-02,2019-08-19,33.0,62.753464,41.743443,83.763489
8,cluster_1,2019-09-09,2019-09-02,31.0,29.879152,9.125082,50.633221
9,cluster_1,2019-09-16,2019-09-02,17.0,28.879152,7.527960,50.230343


In [17]:
# Add the store_id column
p['store_id'] = 'S0094'

# Reorder columns to place store_id at the beginning
p_filtered = p[['store_id'] + [col for col in p.columns if col != 'store_id']]

# Sort values by store_id and unique_id
p_filtered = p_filtered.sort_values(['store_id', 'unique_id'], ascending=True)


# Calculate evaluation metrics
p_filtered['error'] = p_filtered['y'] - p_filtered['AutoARIMA']
p_filtered['squared_error'] = (p_filtered['error']) ** 2
p_filtered['absolute_error'] = np.abs(p_filtered['error'])
p_filtered['absolute_percentage_error'] = p_filtered['absolute_error'] / p_filtered['y']

# Calculate MAE, MSE, and WMAPE at the group level
metrics_summary = p_filtered.groupby('unique_id').agg({
    'absolute_error': 'mean',  # MAE
    'squared_error': 'mean',   # MSE for RMSE calculation
    'y': 'sum',  # Sum of actual values for WMAPE
    'error': 'sum'  # Sum of absolute errors for WMAPE
}).rename(columns={
    'absolute_error': 'MAE',
    'squared_error': 'MSE',
    'y': 'sum_y',
    'error': 'sum_error'
})

# Calculate RMSE from MSE
metrics_summary['RMSE'] = np.sqrt(metrics_summary['MSE'])

# Calculate WMAPE
metrics_summary['WMAPE'] = (metrics_summary['sum_error'].abs() / metrics_summary['sum_y']) * 100

# Drop intermediate columns used for WMAPE calculation
metrics_summary = metrics_summary.drop(columns=['sum_y', 'sum_error'])

# Reset index to make unique_id a column
metrics_summary = metrics_summary.reset_index()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 19, Finished, Available, Finished)

,unique_id,MAE,MSE,RMSE,WMAPE
0,cluster_0,83.069458,9083.411133,95.306931,7.197262
1,cluster_1,10.084488,193.095169,13.895868,26.085707
2,cluster_2,15.590491,344.654633,18.564877,10.886007
3,cluster_3,9.004531,117.315552,10.831230,0.325539
4,cluster_4,26.882284,1169.636353,34.199947,19.514931
5,cluster_5,36.734818,1908.333374,43.684475,3.801085
6,cluster_6,19.125269,465.293457,21.570662,23.466587
7,cluster_7,12.036240,201.749924,14.203870,56.907143
8,cluster_8,30.248892,1142.573853,33.801979,37.800266
9,cluster_9,103.435822,13692.997070,117.017082,36.831902


In [18]:
# save to csv
#p.to_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/predictions_noexogeneous_week_alldates_clusters_CROSSVAL_S0094.csv') 



# Salvando o DataFrame de métricas como CSV no Fabric Lakehouse
metrics_path = 'abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/METRICS_noexogeneous_week_all_dates_clusters_CROSSVAL_S0094.csv'
metrics_summary.to_csv(metrics_path, index=False)

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 20, Finished, Available, Finished)

### From 2018

2017-2018 regists not included --> choose best time intervals to evaluate metrics for ARIMA

In [19]:
# Adjust column names if needed
df_weekly = df_weekly.rename(columns={
    'cluster_id': 'unique_id',
    'date': 'ds',
    'sales': 'y'
})

#funcao do arima
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()


# Rename variable
data = df_weekly.copy()

df_product = data.copy()

# Ensure 'unique_id' and 'ds' are properly formatted
df_product['unique_id'] = df_product['unique_id'].astype(str)
df_product['ds'] = pd.to_datetime(df_product['ds'])

# Separate sales and date from exogenous variables
data2 = df_product[['ds', 'unique_id', 'y']]

# Split data in train for different time-range
train = data2.loc[data2['ds'] >= '2018-01-01']
# Count the occurrences of each product
product_counts = train['unique_id'].value_counts()
# Filter products that appear at least 5 times (to not arise error in crossvalidation)
products_to_keep = product_counts[product_counts >= 5].index
train = train[train['unique_id'].isin(products_to_keep)]

# Define AutoArima model
model = [AutoARIMA(season_length=52)]
#Forecast
sf = StatsForecast(models=model, freq='W-MON', n_jobs=-1)

#Cross-validation
crossvalidation_df = sf.cross_validation(
    df = train,
    h = 2,
    step_size = 2,
    n_windows = 3, 
    level = [90]
  )

print("crossvalidation_df:")
print(crossvalidation_df.head())
p = crossvalidation_df 
p = p.reset_index()
p.rename(columns={'index': 'unique_id'}, inplace=True)


print (p.head())

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 21, Finished, Available, Finished)

crossvalidation_df:
                  ds     cutoff      y   AutoARIMA  AutoARIMA-lo-90  \
unique_id                                                             
cluster_0 2019-08-26 2019-08-19  487.0  352.779266       244.373672   
cluster_0 2019-09-02 2019-08-19  295.0  352.779266       232.079498   
cluster_0 2019-09-09 2019-09-02  413.0  309.211426       198.347351   
cluster_0 2019-09-16 2019-09-02  279.0  332.958649       202.027924   
cluster_0 2019-09-23 2019-09-16  171.0  334.147461       221.589890   

           AutoARIMA-hi-90  
unique_id                   
cluster_0       461.184845  
cluster_0       473.479004  
cluster_0       420.075531  
cluster_0       463.889374  
cluster_0       446.705017  
   unique_id         ds     cutoff      y   AutoARIMA  AutoARIMA-lo-90  \
0  cluster_0 2019-08-26 2019-08-19  487.0  352.779266       244.373672   
1  cluster_0 2019-09-02 2019-08-19  295.0  352.779266       232.079498   
2  cluster_0 2019-09-09 2019-09-02  413.0  309.211426    

In [20]:
# return predictions 
p.head()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 22, Finished, Available, Finished)

,unique_id,ds,cutoff,y,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,cluster_0,2019-08-26,2019-08-19,487.0,352.779266,244.373672,461.184845
1,cluster_0,2019-09-02,2019-08-19,295.0,352.779266,232.079498,473.479004
2,cluster_0,2019-09-09,2019-09-02,413.0,309.211426,198.347351,420.075531
3,cluster_0,2019-09-16,2019-09-02,279.0,332.958649,202.027924,463.889374
4,cluster_0,2019-09-23,2019-09-16,171.0,334.147461,221.589890,446.705017


In [22]:
# Add the store_id column
p['store_id'] = 'S0094'

# Reorder columns to place store_id at the beginning
p_filtered = p[['store_id'] + [col for col in p.columns if col != 'store_id']]

# Sort values by store_id and unique_id
p_filtered = p_filtered.sort_values(['store_id', 'unique_id'], ascending=True)


# Calculate evaluation metrics
p_filtered['error'] = p_filtered['y'] - p_filtered['AutoARIMA']
p_filtered['squared_error'] = (p_filtered['error']) ** 2
p_filtered['absolute_error'] = np.abs(p_filtered['error'])
p_filtered['absolute_percentage_error'] = p_filtered['absolute_error'] / p_filtered['y']

# Calculate MAE, MSE, and WMAPE at the group level
metrics_summary = p_filtered.groupby('unique_id').agg({
    'absolute_error': 'mean',  # MAE
    'squared_error': 'mean',   # MSE for RMSE calculation
    'y': 'sum',  # Sum of actual values for WMAPE
    'error': 'sum'  # Sum of absolute errors for WMAPE
}).rename(columns={
    'absolute_error': 'MAE',
    'squared_error': 'MSE',
    'y': 'sum_y',
    'error': 'sum_error'
})

# Calculate RMSE from MSE
metrics_summary['RMSE'] = np.sqrt(metrics_summary['MSE'])

# Calculate WMAPE
metrics_summary['WMAPE'] = (metrics_summary['sum_error'].abs() / metrics_summary['sum_y']) * 100

# Drop intermediate columns used for WMAPE calculation
metrics_summary = metrics_summary.drop(columns=['sum_y', 'sum_error'])

# Reset index to make unique_id a column
metrics_summary = metrics_summary.reset_index()

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 24, Finished, Available, Finished)

,unique_id,MAE,MSE,RMSE,WMAPE
0,cluster_0,88.507019,10330.612305,101.639618,2.805891
1,cluster_1,9.856942,136.330109,11.676048,39.960579
2,cluster_2,16.846903,396.961945,19.923904,11.928538
3,cluster_3,8.773265,113.178749,10.638550,0.388024
4,cluster_4,28.501907,1138.934692,33.748108,12.350783
5,cluster_5,39.837376,2071.031982,45.508591,8.640121
6,cluster_6,17.122227,382.126495,19.548056,19.869135
7,cluster_7,6.124369,44.226913,6.650332,2.011073
8,cluster_8,30.418570,1095.685181,33.101135,24.176369
9,cluster_9,93.910248,12033.325195,109.696510,38.251556


In [23]:
# save to csv
p.to_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/predictions_noexogeneous_week_from2018_clusters_CROSSVAL_S0094.csv') 



# Salvando o DataFrame de métricas como CSV no Fabric Lakehouse
metrics_path = 'abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/METRICS_noexogeneous_week_all_from2018_clusters_CROSSVAL_S0094.csv'
metrics_summary.to_csv(metrics_path, index=False)

StatementMeta(, 1dcc0977-12c6-4499-80f4-cb7e6b807008, 25, Finished, Available, Finished)